# AutoMix Threshold-Based Router

This notebook implements a simple threshold-based router for the AutoMix model.

## Overview

The router makes routing decisions based on the verifier confidence probability `v` from LM1:
- If `v ≥ threshold t`: Keep LM1's answer
- If `v < threshold t`: Route to LM2

The threshold `t` can be varied to analyze the cost-performance trade-off.


In [2]:
# ============================================================
# Import required libraries
# ============================================================

import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple


## Configuration

Set the dataset name and other parameters here.


In [61]:
# ============================================================
# Configuration
# ============================================================

# Dataset name: cnli_short / coqa_short / narrativeqa_short / qasper_short
DATASET_NAME = "qasper_short"

# Router data path
ROUTER_DATA_PATH = os.path.join("router_data", f"router_data_{DATASET_NAME}.jsonl")

# Output directory for results
OUTPUT_DIR = "threshold_router_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Cost parameters (for cost-performance analysis)
# Cost model: C = C_SLM + C_ver + w * C_LLM
# where C_ver = C_SLM (verification is performed by SLM)
C_SLM = 1.0  # Cost of small language model (LM1)
C_LLM = 20.0  # Cost of large language model (LM2)
C_ver = 4 * C_SLM  # Cost of verification (performed by SLM)

# Cost when keeping LM1: C = C_SLM + C_ver = 2 * C_SLM
# Cost when routing to LM2: C = C_SLM + C_ver + C_LLM = 2 * C_SLM + C_LLM
COST_LM1 = C_SLM + C_ver
COST_LM2 = C_SLM + C_ver + C_LLM

print(f"Dataset: {DATASET_NAME}")
print(f"Router data path: {ROUTER_DATA_PATH}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"\nCost Parameters:")
print(f"  C_SLM: {C_SLM}")
print(f"  C_LLM: {C_LLM}")
print(f"  C_ver: {C_ver}")
print(f"  Cost when keeping LM1: {COST_LM1}")
print(f"  Cost when routing to LM2: {COST_LM2}")


Dataset: qasper_short
Router data path: router_data/router_data_qasper_short.jsonl
Output directory: threshold_router_outputs

Cost Parameters:
  C_SLM: 1.0
  C_LLM: 20.0
  C_ver: 4.0
  Cost when keeping LM1: 5.0
  Cost when routing to LM2: 25.0


## Load Router Data

Load the router data that contains:
- `slm_pred`: LM1's prediction
- `llm_pred`: LM2's prediction
- `slm_confidence`: Verifier confidence probability `v`
- `perf_slm`: Performance of LM1's answer
- `perf_llm`: Performance of LM2's answer


In [62]:
# ============================================================
# Load router data
# ============================================================

if not os.path.exists(ROUTER_DATA_PATH):
    raise FileNotFoundError(f"Router data not found: {ROUTER_DATA_PATH}")

print(f"[INFO] Loading router data from {ROUTER_DATA_PATH}")
df = pd.read_json(ROUTER_DATA_PATH, lines=True, orient="records")

print(f"[INFO] Loaded {len(df)} samples")
print(f"[INFO] Columns: {list(df.columns)}")
print(f"\n[INFO] Confidence value distribution:")
print(df["slm_confidence"].value_counts().sort_index())
print(f"\n[INFO] Sample data:")
df.head()


[INFO] Loading router data from router_data/router_data_qasper_short.jsonl
[INFO] Loaded 1000 samples
[INFO] Columns: ['id', 'dataset', 'gold_output', 'slm_pred', 'llm_pred', 'slm_confidence', 'perf_slm', 'perf_llm']

[INFO] Confidence value distribution:
0.00    225
0.25     16
0.50     14
0.75    745
Name: slm_confidence, dtype: int64

[INFO] Sample data:


,id,dataset,gold_output,slm_pred,llm_pred,slm_confidence,perf_slm,perf_llm
0,753990d0b621d390ed58f20c4d9e4f065f0dc672,qasper_short,a vocabulary of positive and negative predicat...,A seed lexicon consisting of 15 positive words...,a small list of 30 Japanese emotion predicates...,0.75,0.357143,0.256410
1,753990d0b621d390ed58f20c4d9e4f065f0dc672,qasper_short,seed lexicon consists of positive and negative...,A seed lexicon consisting of 15 positive words...,a list of 15 positive and 15 negative Japanese...,0.75,0.600000,0.357143
2,9d578ddccc27dd849244d632dd0f6bf27348ad81,qasper_short,Using all data to train: AL -- BiGRU achieved ...,"The proposed method performed well, even with ...",BiGRU trained with AL+CA+CO achieved the highe...,0.75,0.090090,0.105263
3,02e4bf719b1a504e385c35c6186742e720bcb281,qasper_short,"based on the relation between events, the sugg...",The answer is through the exploitation of disc...,By exploiting discourse relations: for Cause p...,0.75,0.176471,0.318182
4,02e4bf719b1a504e385c35c6186742e720bcb281,qasper_short,cause relation: both events in the relation sh...,The relations are used to propagate polarity t...,By using discourse relations between event pai...,0.75,0.222222,0.355556


## Threshold-Based Router Function

Implement the simple threshold-based routing logic.


In [24]:
# ============================================================
# Threshold-based router function
# ============================================================

def apply_threshold_router(df: pd.DataFrame, threshold: float) -> Tuple[pd.DataFrame, Dict]:
    """
    Apply threshold-based router to route queries between LM1 and LM2.
    
    Args:
        df: DataFrame containing router data
        threshold: Threshold value t. If verifier confidence v >= t, use LM1, else route to LM2.
    
    Returns:
        results_df: DataFrame with routing results
        stats: Dictionary with summary statistics
    """
    results = []
    lm1_count = 0
    lm2_count = 0
    
    for i, row in df.iterrows():
        v = float(row["slm_confidence"])  # Verifier confidence
        slm_pred = row["slm_pred"]  # LM1's answer
        llm_pred = row["llm_pred"]  # LM2's answer
        
        # Routing decision: if v >= threshold, use LM1, else route to LM2
        # Cost model: C = C_SLM + C_ver + w * C_LLM
        if v >= threshold:
            final_ans = slm_pred
            action = "keep"  # Keep LM1's answer
            model_used = "lm1"
            lm1_count += 1
            cost = COST_LM1  # C_SLM + C_ver = 2 * C_SLM
        else:
            final_ans = llm_pred
            action = "route"  # Route to LM2
            model_used = "lm2"
            lm2_count += 1
            cost = COST_LM2  # C_SLM + C_ver + C_LLM = 2 * C_SLM + C_LLM
        
        # Compute performance for the chosen answer
        perf_chosen = row["perf_slm"] if model_used == "lm1" else row["perf_llm"]
        
        results.append({
            "id": row.get("id", i),
            "dataset": row.get("dataset", DATASET_NAME),
            "slm_confidence": v,
            "threshold": threshold,
            "action": action,
            "model_used": model_used,
            "gold_answer": row.get("gold_output"),
            "final_answer": final_ans,
            "slm_pred": slm_pred,
            "llm_pred": llm_pred,
            "perf_chosen": perf_chosen,
            "perf_slm": row.get("perf_slm"),
            "perf_llm": row.get("perf_llm"),
            "cost": cost,
        })
    
    results_df = pd.DataFrame(results)
    
    # Compute summary statistics
    total = len(results_df)
    lm1_ratio = lm1_count / total
    lm2_ratio = lm2_count / total
    avg_perf = results_df["perf_chosen"].mean()
    avg_perf_slm = results_df["perf_slm"].mean()
    avg_perf_llm = results_df["perf_llm"].mean()
    avg_cost = results_df["cost"].mean()
    total_cost = results_df["cost"].sum()
    
    stats = {
        "threshold": threshold,
        "total": total,
        "lm1_count": lm1_count,
        "lm2_count": lm2_count,
        "lm1_ratio": lm1_ratio,
        "lm2_ratio": lm2_ratio,
        "avg_perf": avg_perf,
        "avg_perf_slm": avg_perf_slm,
        "avg_perf_llm": avg_perf_llm,
        "avg_cost": avg_cost,
        "total_cost": total_cost,
    }
    
    return results_df, stats




In [29]:
# ============================================================
# Test with a single threshold
# ============================================================

THRESHOLD = 0.5  # Adjust this threshold value

print(f"[INFO] Testing threshold = {THRESHOLD}")
results_df, stats = apply_threshold_router(df, THRESHOLD)

# Display results
print(f"\n{'='*60}")
print(f"Threshold Router Results for {DATASET_NAME}")
print(f"{'='*60}")
print(f"Threshold: {stats['threshold']:.3f}")
print(f"Total samples: {stats['total']}")
print(f"LM1 (keep): {stats['lm1_count']} ({stats['lm1_ratio']:.2%})")
print(f"LM2 (route): {stats['lm2_count']} ({stats['lm2_ratio']:.2%})")
print(f"\nPerformance:")
print(f"  Average performance (routed): {stats['avg_perf']:.4f}")
print(f"  Average performance (LM1 only): {stats['avg_perf_slm']:.4f}")
print(f"  Average performance (LM2 only): {stats['avg_perf_llm']:.4f}")
print(f"\nCost:")
print(f"  Average cost per sample: {stats['avg_cost']:.2f}")
print(f"  Total cost: {stats['total_cost']:.2f}")
print(f"{'='*60}\n")

# Show sample results
print("Sample results:")
results_df[["slm_confidence", "threshold", "action", "model_used", "perf_chosen", "cost"]].head(10)


[INFO] Testing threshold = 0.5

Threshold Router Results for coqa_short
Threshold: 0.500
Total samples: 1000
LM1 (keep): 668 (66.80%)
LM2 (route): 332 (33.20%)

Performance:
  Average performance (routed): 0.5256
  Average performance (LM1 only): 0.4626
  Average performance (LM2 only): 0.5939

Cost:
  Average cost per sample: 8.64
  Total cost: 8640.00

Sample results:


,slm_confidence,threshold,action,model_used,perf_chosen,cost
0,0.75,0.5,keep,lm1,1.000000,2.0
1,0.75,0.5,keep,lm1,0.400000,2.0
2,0.75,0.5,keep,lm1,0.142857,2.0
3,0.75,0.5,keep,lm1,0.833333,2.0
4,0.75,0.5,keep,lm1,0.500000,2.0
5,0.75,0.5,keep,lm1,0.000000,2.0
6,0.75,0.5,keep,lm1,0.500000,2.0
7,0.75,0.5,keep,lm1,0.400000,2.0
8,0.75,0.5,keep,lm1,1.000000,2.0
9,0.75,0.5,keep,lm1,0.888889,2.0


In [30]:
# Save results for single threshold
output_filename = f"threshold_{THRESHOLD:.2f}_{DATASET_NAME}.json"
output_path = os.path.join(OUTPUT_DIR, output_filename)

results_dict = results_df.to_dict(orient="records")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results_dict, f, ensure_ascii=False, indent=2)

print(f"Results saved to: {output_path}")


Results saved to: threshold_router_outputs/threshold_0.50_coqa_short.json


## Threshold Sweep Analysis

Analyze the cost-performance trade-off by testing multiple threshold values.


## IBC (Incremental Benefit Per Cost) Metric

The IBC metric quantifies the cost-effectiveness of routing methods relative to the SLM baseline.

**Cost Model:**
- Cost when keeping LM1: `C = C_SLM + C_ver = 2 * C_SLM`
- Cost when routing to LM2: `C = C_SLM + C_ver + C_LLM = 2 * C_SLM + C_LLM`
- Where `C_ver = C_SLM` (verification is performed by SLM)

**IBC Formulas:**
- **IBC_M** = (P_M - P_SLM) / (C_M - C_SLM) - Incremental benefit per cost for method M
- **IBC_BASE** = (P_LLM - P_SLM) / (C_LLM - C_SLM) - Baseline (always using LLM)
- **Δ_IBC(M)** = ((IBC_M - IBC_BASE) / IBC_BASE) * 100 - Percentage improvement over baseline

**Interpretation:**
- Positive Δ_IBC: Method is more cost-effective than always using LLM
- Negative Δ_IBC: Method is less cost-effective than always using LLM
- Higher IBC values indicate better cost-effectiveness


In [63]:
# ============================================================
# Threshold sweep: test multiple thresholds
# ============================================================

# Define thresholds to test
# Common values: 0.0, 0.25, 0.5, 0.75, 1.0
THRESHOLDS = [0.25, 0.5, 0.75]

# Or create a finer grid
# THRESHOLDS = np.arange(0.0, 1.01, 0.1).tolist()

print(f"[INFO] Analyzing threshold sweep for {DATASET_NAME}")
print(f"[INFO] Testing {len(THRESHOLDS)} thresholds: {THRESHOLDS}\n")

sweep_results = []
all_results = []

for threshold in THRESHOLDS:
    print(f"Processing threshold = {threshold:.3f}...")
    results_df, stats = apply_threshold_router(df, threshold)
    sweep_results.append(stats)
    all_results.append(results_df)
    
    # Save results for single threshold
    output_filename = f"threshold_{threshold:.2f}_{DATASET_NAME}.json"
    output_path = os.path.join(OUTPUT_DIR, output_filename)

    results_dict = results_df.to_dict(orient="records")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results_dict, f, ensure_ascii=False, indent=2)

    # Print quick summary
    print(f"  LM1: {stats['lm1_ratio']:.2%}, LM2: {stats['lm2_ratio']:.2%}, "
          f"Perf: {stats['avg_perf']:.4f}, Avg Cost: {stats['avg_cost']:.2f}")

# Convert to DataFrame for easier analysis
sweep_df = pd.DataFrame(sweep_results)

print(f"\n{'='*60}")
print(f"Threshold Sweep Summary for {DATASET_NAME}")
print(f"{'='*60}")
print(sweep_df[["threshold", "lm1_ratio", "lm2_ratio", "avg_perf", "avg_cost", "total_cost"]].to_string(index=False))
print(f"{'='*60}\n")


[INFO] Analyzing threshold sweep for qasper_short
[INFO] Testing 3 thresholds: [0.25, 0.5, 0.75]

Processing threshold = 0.250...
  LM1: 77.50%, LM2: 22.50%, Perf: 0.3359, Avg Cost: 9.50
Processing threshold = 0.500...
  LM1: 75.90%, LM2: 24.10%, Perf: 0.3399, Avg Cost: 9.82
Processing threshold = 0.750...
  LM1: 74.50%, LM2: 25.50%, Perf: 0.3421, Avg Cost: 10.10

Threshold Sweep Summary for qasper_short
 threshold  lm1_ratio  lm2_ratio  avg_perf  avg_cost  total_cost
      0.25      0.775      0.225  0.335881      9.50      9500.0
      0.50      0.759      0.241  0.339856      9.82      9820.0
      0.75      0.745      0.255  0.342115     10.10     10100.0



In [64]:
# ============================================================
# Calculate IBC (Incremental Benefit Per Cost) metrics
# ============================================================
# Based on the IBC metric defined in the paper:
# IBC_M = (P_M - P_SLM) / (C_M - C_SLM)
# IBC_BASE = (P_LLM - P_SLM) / (C_LLM - C_SLM)
# Δ_IBC(M) = ((IBC_M - IBC_BASE) / IBC_BASE) * 100

# Get baseline performance values
P_SLM = sweep_df["avg_perf_slm"].iloc[0]  # Average performance of SLM (with verification)
P_LLM = sweep_df["avg_perf_llm"].iloc[0]  # Average performance of LLM (with verification)

# Baseline costs
# C_SLM = C_SLM + C_ver = 2 * C_SLM (SLM inference + verification)
# C_LLM = C_SLM + C_ver + C_LLM = 2 * C_SLM + C_LLM (SLM inference + verification + LLM inference)
C_SLM_baseline = COST_LM1  # Cost of SLM method: 2 * C_SLM
C_LLM_baseline = COST_LM2  # Cost of LLM method: 2 * C_SLM + C_LLM

# Calculate IBC_BASE: Baseline incremental benefit per cost
# IBC_BASE represents the benefit of always using LLM over SLM
# IBC_BASE = (P_LLM - P_SLM) / (C_LLM - C_SLM)
IBC_BASE = (P_LLM - P_SLM) / (C_LLM_baseline - C_SLM_baseline)

print(f"\n{'='*60}")
print(f"IBC (Incremental Benefit Per Cost) Baseline")
print(f"{'='*60}")
print(f"P_SLM (SLM performance): {P_SLM:.4f}")
print(f"P_LLM (LLM performance): {P_LLM:.4f}")
print(f"C_SLM (SLM cost): {C_SLM_baseline:.2f}")
print(f"C_LLM (LLM cost): {C_LLM_baseline:.2f}")
print(f"IBC_BASE: {IBC_BASE:.6f}")
print(f"{'='*60}\n")

# Calculate IBC for each threshold method M
# IBC_M = (P_M - P_SLM) / (C_M - C_SLM)
# where P_M is the performance of method M, C_M is the cost of method M
sweep_df["IBC"] = (sweep_df["avg_perf"] - P_SLM) / (sweep_df["avg_cost"] - C_SLM_baseline)

# Calculate Δ_IBC: Percentage lift of IBC
# Δ_IBC(M) = ((IBC_M - IBC_BASE) / IBC_BASE) * 100
# Positive values indicate the method achieves performance increments more cost-effectively than baseline
# Negative values indicate reduced efficiency compared to baseline
sweep_df["delta_IBC"] = ((sweep_df["IBC"] - IBC_BASE) / IBC_BASE) * 100



# Display the sweep results with IBC metrics
print(f"\n{'='*60}")
print(f"Threshold Sweep Summary with IBC Metrics for {DATASET_NAME}")
print(f"{'='*60}")
display_cols = ["threshold", "lm1_ratio", "lm2_ratio", "avg_perf", "avg_cost", "IBC", "delta_IBC"]
print(sweep_df[display_cols].round(6).to_string(index=False))
print(f"{'='*60}\n")

# Display interpretation
print("IBC Metric Interpretation:")
print(f"  IBC_BASE (baseline): {IBC_BASE:.6f} - Benefit of always using LLM over SLM")
print(f"  Positive Δ_IBC: Method is more cost-effective than baseline")
print(f"  Negative Δ_IBC: Method is less cost-effective than baseline")
print()

# Save sweep summary
sweep_summary_path = os.path.join(OUTPUT_DIR, f"threshold_sweep_summary_{DATASET_NAME}.json")
sweep_summary_with_ibc = sweep_df.to_dict(orient="records")
with open(sweep_summary_path, "w", encoding="utf-8") as f:
    json.dump(sweep_summary_with_ibc, f, ensure_ascii=False, indent=2)

print(f"Sweep summary saved to: {sweep_summary_path}")

sweep_df



IBC (Incremental Benefit Per Cost) Baseline
P_SLM (SLM performance): 0.3019
P_LLM (LLM performance): 0.3853
C_SLM (SLM cost): 5.00
C_LLM (LLM cost): 25.00
IBC_BASE: 0.004166


Threshold Sweep Summary with IBC Metrics for qasper_short
 threshold  lm1_ratio  lm2_ratio  avg_perf  avg_cost      IBC  delta_IBC
      0.25      0.775      0.225  0.335881      9.50 0.007543  81.035751
      0.50      0.759      0.241  0.339856      9.82 0.007867  88.810684
      0.75      0.745      0.255  0.342115     10.10 0.007878  89.072782

IBC Metric Interpretation:
  IBC_BASE (baseline): 0.004166 - Benefit of always using LLM over SLM
  Positive Δ_IBC: Method is more cost-effective than baseline
  Negative Δ_IBC: Method is less cost-effective than baseline

Sweep summary saved to: threshold_router_outputs/threshold_sweep_summary_qasper_short.json


,threshold,total,lm1_count,lm2_count,lm1_ratio,lm2_ratio,avg_perf,avg_perf_slm,avg_perf_llm,avg_cost,total_cost,IBC,delta_IBC
0,0.25,1000,775,225,0.775,0.225,0.335881,0.301938,0.385268,9.50,9500.0,0.007543,81.035751
1,0.50,1000,759,241,0.759,0.241,0.339856,0.301938,0.385268,9.82,9820.0,0.007867,88.810684
2,0.75,1000,745,255,0.745,0.255,0.342115,0.301938,0.385268,10.10,10100.0,0.007878,89.072782


## IBC-Based Optimal Threshold Analysis

Additional analysis focused on IBC (Incremental Benefit Per Cost) metrics for identifying the most cost-effective thresholds.


In [65]:
# ============================================================
# IBC-based optimal threshold analysis
# ============================================================

print("="*60)
print("IBC-Based Optimal Threshold Analysis")
print("="*60)

# 1. Maximum IBC (Best cost-effectiveness relative to SLM baseline)
max_ibc_idx = sweep_df["IBC"].idxmax()
max_ibc_threshold = sweep_df.loc[max_ibc_idx, "threshold"]
max_ibc_value = sweep_df.loc[max_ibc_idx, "IBC"]
max_ibc_delta = sweep_df.loc[max_ibc_idx, "delta_IBC"]
max_ibc_perf = sweep_df.loc[max_ibc_idx, "avg_perf"]
max_ibc_cost = sweep_df.loc[max_ibc_idx, "avg_cost"]
print(f"\n1. Maximum IBC (Best Cost-Effectiveness vs SLM):")
print(f"   Threshold: {max_ibc_threshold:.3f}")
print(f"   IBC: {max_ibc_value:.6f}")
print(f"   Δ_IBC: {max_ibc_delta:.2f}% (vs baseline)")
print(f"   Performance: {max_ibc_perf:.4f}")
print(f"   Cost: {max_ibc_cost:.2f}")

# 2. Maximum Δ_IBC (Best improvement over baseline LLM)
max_delta_ibc_idx = sweep_df["delta_IBC"].idxmax()
max_delta_ibc_threshold = sweep_df.loc[max_delta_ibc_idx, "threshold"]
max_delta_ibc_value = sweep_df.loc[max_delta_ibc_idx, "delta_IBC"]
max_delta_ibc_ibc = sweep_df.loc[max_delta_ibc_idx, "IBC"]
max_delta_ibc_perf = sweep_df.loc[max_delta_ibc_idx, "avg_perf"]
max_delta_ibc_cost = sweep_df.loc[max_delta_ibc_idx, "avg_cost"]
print(f"\n2. Maximum Δ_IBC (Best Improvement over Baseline LLM):")
print(f"   Threshold: {max_delta_ibc_threshold:.3f}")
print(f"   Δ_IBC: {max_delta_ibc_value:.2f}%")
print(f"   IBC: {max_delta_ibc_ibc:.6f}")
print(f"   Performance: {max_delta_ibc_perf:.4f}")
print(f"   Cost: {max_delta_ibc_cost:.2f}")

# 3. Positive Δ_IBC methods (more cost-effective than baseline)
positive_delta = sweep_df[sweep_df["delta_IBC"] > 0]
if len(positive_delta) > 0:
    print(f"\n3. Methods with Positive Δ_IBC (More Cost-Effective than Baseline):")
    print(f"   Count: {len(positive_delta)} out of {len(sweep_df)}")
    print(f"   Best threshold: {positive_delta.loc[positive_delta['delta_IBC'].idxmax(), 'threshold']:.3f}")
    print(f"   Best Δ_IBC: {positive_delta['delta_IBC'].max():.2f}%")
else:
    print(f"\n3. Methods with Positive Δ_IBC:")
    print(f"   None - All methods are less cost-effective than baseline")

print("\n" + "="*60)

# Display summary table with IBC metrics
print("\nSummary Table with IBC Metrics:")
summary_cols = ["threshold", "avg_perf", "avg_cost", "IBC", "delta_IBC"]
print(sweep_df[summary_cols].round(6).to_string(index=False))


IBC-Based Optimal Threshold Analysis

1. Maximum IBC (Best Cost-Effectiveness vs SLM):
   Threshold: 0.750
   IBC: 0.007878
   Δ_IBC: 89.07% (vs baseline)
   Performance: 0.3421
   Cost: 10.10

2. Maximum Δ_IBC (Best Improvement over Baseline LLM):
   Threshold: 0.750
   Δ_IBC: 89.07%
   IBC: 0.007878
   Performance: 0.3421
   Cost: 10.10

3. Methods with Positive Δ_IBC (More Cost-Effective than Baseline):
   Count: 3 out of 3
   Best threshold: 0.750
   Best Δ_IBC: 89.07%


Summary Table with IBC Metrics:
 threshold  avg_perf  avg_cost      IBC  delta_IBC
      0.25  0.335881      9.50 0.007543  81.035751
      0.50  0.339856      9.82 0.007867  88.810684
      0.75  0.342115     10.10 0.007878  89.072782
